### Prepare the Home Value dataset
#### By Gen Ho, Xinqian Zhai, Cliff Gong

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import os
from datetime import datetime 
from datetime import timedelta

In [2]:
def get_home_value_data():

    df_home_value = pd.read_csv("./data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
    
    date_range = df_home_value.columns.values[9:]
    date_range = [x for x in date_range if x >='2017-12-31']
    df_home_value = df_home_value[['State', 'RegionName'] + date_range]
    df_home_value = df_home_value[(df_home_value['State']=="CA") | (df_home_value['State']=="NY") | (df_home_value['State']=="TX")]
    # Here we add 1 day to each data column name to make it the start of each month.
    # This will help in subsequent data joining 
    df_home_value.columns = ['state', 'county']  + [(datetime.strptime(x, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')  for x in date_range]
    df_home_value['county'] = df_home_value['county'] + [','] + df_home_value['state']
    df_home_value = df_home_value.drop(columns='state')
    df_home_value = df_home_value.reset_index(drop=True)
    
    return df_home_value

In [3]:
# # to get the size of dataset
# df_home_value = pd.read_csv("./data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
# df_home_value.shape

In [4]:
get_home_value_data().head()

,county,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,...,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01
0,"Los Angeles County,CA",602936.0,607524.0,612344.0,617458.0,621633.0,624114.0,626312.0,628704.0,629633.0,...,687998.0,694954.0,701797.0,708899.0,717023.0,725546.0,736602.0,753097.0,772166.0,790721.0
1,"Harris County,TX",186483.0,187064.0,187625.0,188326.0,189389.0,190504.0,191454.0,192214.0,193188.0,...,211682.0,213753.0,215825.0,218345.0,221050.0,223181.0,225732.0,228876.0,233548.0,238585.0
2,"San Diego County,CA",571433.0,577995.0,583902.0,587469.0,586869.0,586151.0,587194.0,589360.0,591832.0,...,656884.0,667567.0,678553.0,690090.0,701775.0,713657.0,728400.0,747896.0,770632.0,792531.0
3,"Orange County,CA",718048.0,721360.0,724754.0,726842.0,731361.0,734774.0,738855.0,739600.0,739812.0,...,784988.0,793470.0,804174.0,814515.0,824504.0,834751.0,848409.0,868412.0,893058.0,916135.0
4,"Kings County,NY",634737.0,637338.0,641380.0,643817.0,645507.0,647279.0,650709.0,653965.0,656838.0,...,680890.0,683737.0,685777.0,687419.0,688786.0,689305.0,690547.0,691107.0,693181.0,694832.0


### Prepare the Rental dataset

In [5]:
def get_rental_data():

    df_rental = pd.read_csv("./data/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv")
    
    date_range = df_rental.columns.values[4:]
    date_range = [x for x in date_range if x >= '2018-01']

    df_rental['state'] = df_rental['MsaName'].str[-2:]
    df_rental = df_rental[(df_rental['state']=='CA') | (df_rental['state']=='NY') | (df_rental['state']=='TX')] 
    
    df_rental = df_rental[['RegionName', 'state'] + date_range]
    df_rental['RegionName'] = df_rental['RegionName'].astype(str)
    df_rental.columns = ['zip', 'state'] + [x + '-01' for x in date_range]
    df_rental = df_rental.reset_index(drop=True)

    return df_rental

In [6]:
# check the data size
# df_rental = pd.read_csv("./data/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv")
# df_rental.info()

In [7]:
get_rental_data().head()

,zip,state,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,...,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01
0,10025,NY,3152.0,3154.0,3156.0,3157.0,3163.0,3168.0,3173.0,3180.0,...,2954.0,2944.0,2934.0,2924.0,2917.0,2909.0,2902.0,2899.0,2897.0,2894.0
1,10023,NY,3163.0,3164.0,3166.0,3167.0,3171.0,3174.0,3177.0,3179.0,...,2949.0,2937.0,2925.0,2913.0,2904.0,2894.0,2885.0,2880.0,2874.0,2869.0
2,77494,TX,1323.0,1325.0,1327.0,1329.0,1330.0,1330.0,1331.0,1331.0,...,1378.0,1387.0,1395.0,1404.0,1414.0,1423.0,1432.0,1442.0,1452.0,1462.0
3,77449,TX,1323.0,1327.0,1331.0,1335.0,1338.0,1340.0,1343.0,1345.0,...,1438.0,1452.0,1466.0,1479.0,1494.0,1508.0,1522.0,1537.0,1552.0,1567.0
4,10002,NY,3075.0,3081.0,3087.0,3093.0,3100.0,3107.0,3114.0,3118.0,...,2852.0,2838.0,2824.0,2810.0,2800.0,2789.0,2779.0,2774.0,2768.0,2763.0


### Prepare the ZipCode Mapping dataset

In [8]:
def get_zipcode_data():

    df_zipcode = pd.read_csv("./data/zip_code_database.csv")
    df_zipcode = df_zipcode[['zip', 'state', 'county']]
    df_zipcode = df_zipcode[(df_zipcode['state'] == 'CA') | (df_zipcode['state'] == 'NY') | (df_zipcode['state'] == 'TX')]
    df_zipcode['zip'] = df_zipcode['zip'].astype(str)
    df_zipcode = df_zipcode.reset_index(drop=True)
    
    return df_zipcode

In [9]:
# df_zipcode = pd.read_csv("./data/zip_code_database.csv")
# df_zipcode.shape

In [10]:
get_zipcode_data().head()

,zip,state,county
0,501,NY,Suffolk County
1,544,NY,Suffolk County
2,6390,NY,Suffolk County
3,10001,NY,New York County
4,10002,NY,New York County


### Prepare Mortgage Rate dataset

In [11]:
# !pip install xlrd  # for read the excel file in deepnote, already installed
# pip install --upgrade pip # upgraded the pip version to install the xlrd, already upgraded

def get_rate_data():
 
    df_rate = pd.read_excel('./data/MORTGAGE30US.xls', header = 10)
    
    df_rate.columns = ['date', 'rate']
    # convert datetime to str since the date dtype in excel is datetime64 instead of str
    df_rate['date'] = df_rate['date'].astype(str)
    
    df_rate = df_rate[df_rate['date'] > '2017-12-31']
    # The rate records are created on every Thursday 
    # Here we only pick the 1st record of each month
    df_rate['year_month'] = df_rate['date'].str[0:7]
    df_rate = df_rate.groupby(by='year_month').first().reset_index(drop=True)
    # We change the date to 1st day of a month for easier joining with other datasets 
    df_rate['date'] = df_rate['date'].str[0:8] + '01'
    #df_rate = df_rate.reset_index(drop=True)
    df_rate['rate'] = df_rate['rate']/100
    df_rate = df_rate.set_index('date')

    return df_rate

In [12]:
# df_rate = pd.read_csv('./data/MORTGAGE30US.csv', header = 10)
# df_rate.shape

In [13]:
get_rate_data().head()

,rate
date,
2018-01-01,0.0395
2018-02-01,0.0422
2018-03-01,0.0443
2018-04-01,0.0440
2018-05-01,0.0455


### Prepare Property Tax Rate dataset

In [14]:
def get_property_tax_data():

    df_ca = pd.read_csv('./data/PropertyTaxRateCA.csv')
    df_ca.columns = ['county', 'rate']
    df_ca['county'] = df_ca['county'] + ",CA"
    #df_ca['state'] = 'CA'

    df_ny = pd.read_csv('./data/PropertyTaxRateNY.csv')
    df_ny.columns = ['county', 'rate']
    #df_ny['state'] = 'NY'
    df_ny['county'] = df_ny['county'] + ' County,NY'

    df_tx = pd.read_csv('./data/PropertyTaxRateTX.csv')
    df_tx.columns = ['county', 'rate']
    #df_tx['state'] = 'TX'
    df_tx['county'] = df_tx['county'] + ' County,TX'

    df_all = df_ca.append(df_ny, ignore_index=True).append(df_tx, ignore_index=True)
    df_all = df_all.set_index('county')

    return df_all

In [15]:
# get_property_tax_data().shape

In [16]:
get_property_tax_data().head()

,rate
county,
"Alameda County,CA",0.0078
"Alpine County,CA",0.0082
"Amador County,CA",0.0074
"Butte County,CA",0.0074
"Calaveras County,CA",0.0081


### Get all required datasets

In [17]:
df_home_value = get_home_value_data()
df_rental = get_rental_data()
df_rate = get_rate_data()
df_zipcode = get_zipcode_data()
df_property_tax = get_property_tax_data()

Join the df_rental and df_zipcode by zip code

In [18]:
df_rental_with_county = df_rental.merge(df_zipcode, left_on='zip', right_on='zip', how='left')
df_rental_with_county = df_rental_with_county.drop(columns=['zip', 'state_y'])
df_rental_with_county = df_rental_with_county.rename(columns={'state_x':'state'})

# The rental data is on zip code level, means there can be more than 1 record per county.
# To tackle this we will group the records by state and county, then take average for each rent
df_rental_with_county = df_rental_with_county.groupby(by=['state', 'county']).median() # use median instead of mean to remove effects from extreme values 

In [19]:
df_rental_with_county = df_rental.merge(df_zipcode, left_on='zip', right_on='zip', how='left')
df_rental_with_county = df_rental_with_county.drop(columns=['zip', 'state_y'])
df_rental_with_county = df_rental_with_county.rename(columns={'state_x':'state'})

# The rental data is on zip code level, means there can be more than 1 record per county.
# To tackle this we will group the records by state and county, then take average for each rent
df_rental_with_county = df_rental_with_county.groupby(by=['state', 'county']).median() # use median instead of mean to remove effects from extreme values 

# Also need to interpolate for NaN values in the dataset
# Can check with this:  df_rental_with_county.isnull().any(axis=1)
df_rental_with_county  = df_rental_with_county.interpolate(axis=1).reset_index().round(0)

df_rental_with_county['county'] = df_rental_with_county['county'] + ',' + df_rental_with_county['state']
df_rental_with_county = df_rental_with_county.drop(columns='state')

In [20]:
df_rental_with_county.head()

,county,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,...,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01
0,"Alameda County,CA",2541.0,2545.0,2550.0,2555.0,2560.0,2566.0,2571.0,2577.0,2582.0,...,2583.0,2582.0,2582.0,2581.0,2580.0,2579.0,2578.0,2579.0,2579.0,2579.0
1,"Contra Costa County,CA",2468.0,2474.0,2480.0,2487.0,2491.0,2496.0,2501.0,2504.0,2507.0,...,2548.0,2562.0,2533.0,2545.0,2557.0,2569.0,2631.0,2593.0,2605.0,2676.0
2,"Kern County,CA",953.0,957.0,961.0,964.0,969.0,974.0,979.0,985.0,990.0,...,1134.0,1144.0,1154.0,1164.0,1174.0,1184.0,1194.0,1204.0,1215.0,1225.0
3,"Los Angeles County,CA",2181.0,2191.0,2198.0,2205.0,2213.0,2222.0,2232.0,2239.0,2252.0,...,2359.0,2363.0,2372.0,2386.0,2387.0,2398.0,2400.0,2423.0,2424.0,2432.0
4,"Marin County,CA",3312.0,3324.0,3337.0,3350.0,3362.0,3374.0,3385.0,3394.0,3403.0,...,3529.0,3536.0,3541.0,3548.0,3554.0,3560.0,3566.0,3572.0,3578.0,3584.0


There are 8 counties in TX which has null values. After cross checked with Rental dataset, found these 8 counties are not covered in the Rental dataset.  So, we will just remove them 

In [21]:
df_home_value[df_home_value.isnull().any(axis=1)]

,county,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,...,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01
110,"Victoria County,TX",NaN,NaN,NaN,149801.0,149216.0,148703.0,147773.0,147098.0,146772.0,...,159894.0,161431.0,163292.0,165044.0,166647.0,168108.0,169795.0,171961.0,174805.0,177449.0
186,"Hopkins County,TX",NaN,125598.0,125761.0,125887.0,126137.0,126357.0,126615.0,126450.0,126167.0,...,133701.0,134238.0,135573.0,136207.0,136982.0,137730.0,138451.0,138829.0,139187.0,139345.0
296,"Yoakum County,TX",NaN,NaN,NaN,104670.0,105207.0,105667.0,106475.0,107133.0,107744.0,...,125168.0,126255.0,127300.0,128317.0,129355.0,130441.0,131857.0,133433.0,135131.0,136602.0
305,"Garza County,TX",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85657.0,...,78642.0,78631.0,78381.0,77823.0,78120.0,78336.0,78328.0,78352.0,78282.0,78480.0
310,"Lynn County,TX",NaN,NaN,NaN,NaN,NaN,NaN,139988.0,141142.0,141866.0,...,150150.0,151080.0,151891.0,152600.0,153106.0,153899.0,154938.0,155898.0,156771.0,157645.0
312,"Hansford County,TX",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96167.0,95616.0,95101.0,94847.0,94517.0,94064.0,93836.0,93533.0,93743.0,93991.0
321,"Baylor County,TX",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80977.0,81666.0,82436.0,83199.0,84093.0,85041.0,86219.0,87487.0,88769.0,89975.0
322,"Knox County,TX",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61543.0,61755.0,61978.0,62209.0,62449.0,62697.0,62952.0,63219.0,63527.0,63805.0


In [22]:
df_home_value = df_home_value.dropna(axis=0)
df_home_value[df_home_value.isnull().any(axis=1)]

,county,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,...,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01


### Generate Returns 

In [23]:
df_home_value = df_home_value.set_index('county').T 
df_rental_with_county = df_rental_with_county.set_index('county').T

#For the Home Value dataset, we only need the counties which exist in Rental dataset
df_home_value = df_home_value[df_rental_with_county.columns.values]

In [24]:
dates = df_rental_with_county.index.values
counties = df_rental_with_county.columns.values 

# Create a dataframe for returns. This dataframe will have same size as df_rental_with_county 
n_row, n_col = df_rental_with_county.shape
returns = np.zeros((n_row, n_col), dtype=float)
df_returns = pd.DataFrame(returns, columns=df_rental_with_county.columns.values, index=df_rental_with_county.index.values)

In [25]:
for d in dates:
    for c in counties:
        
        # find out the gross_rental_income
        monthly_rental_price = df_rental_with_county.loc[d, c]
        gross_rental_income = monthly_rental_price * 12

        selling_price = df_home_value.loc[d, c]

        # find out the expenses 
        interest_rate = df_rate.loc[d, 'rate'] 
        mortgate_payment_yearly = (selling_price * 0.75) * interest_rate 
        property_tax_rate = df_property_tax.loc[c, 'rate']
        property_tax_yearly = selling_price * property_tax_rate   
        maintenance_yearly = gross_rental_income * 0.10


        #print("mortgate_payment_yearly:{}, property_tax_yearly:{}, maintenance_yearly:{}".format(mortgate_payment_yearly, property_tax_yearly, maintenance_yearly))
        expenses = mortgate_payment_yearly + property_tax_yearly + maintenance_yearly
        
        # find out the initial cash out of pocket 
        down_payment = selling_price * (1 - 0.75)
        closing_cost = selling_price * 0.05
        remodel_cost = 10000
        initial_out_of_pocket = down_payment + closing_cost + remodel_cost

        # The return value is a percentage 
        cash_on_cash_return = (gross_rental_income - expenses) / initial_out_of_pocket * 100

        #print("date:{}, county:{}, gross_rental_income:{}, expenses:{}, initial_out_of_pocket:{}, cash_on_cash_return:{}".format(d, c, gross_rental_income, expenses, initial_out_of_pocket, cash_on_cash_return))

        df_returns.loc[d, c] = round(cash_on_cash_return , 2)

In [26]:
df_returns.head()

,"Alameda County,CA","Contra Costa County,CA","Kern County,CA","Los Angeles County,CA","Marin County,CA","Orange County,CA","Placer County,CA","Riverside County,CA","Sacramento County,CA","San Bernardino County,CA",...,"Guadalupe County,TX","Harris County,TX","Hays County,TX","Johnson County,TX","Kaufman County,TX","Montgomery County,TX","Rockwall County,TX","Tarrant County,TX","Travis County,TX","Williamson County,TX"
2018-01-01,-1.53,0.18,2.64,0.71,-2.17,-0.64,1.88,4.14,3.22,5.78,...,5.45,8.29,-0.82,9.43,8.90,3.70,4.86,6.23,-1.39,-0.58
2018-02-01,-2.32,-0.55,2.07,0.03,-2.83,-1.23,1.33,3.55,2.57,5.12,...,4.87,7.70,-1.39,8.69,8.31,3.17,3.18,5.64,-2.01,-1.21
2018-03-01,-2.91,-1.12,1.64,-0.52,-3.29,-1.88,0.81,3.01,2.04,4.63,...,4.39,7.25,-1.83,8.08,7.84,2.82,2.67,5.15,-2.49,-1.64
2018-04-01,-2.89,-1.09,1.71,-0.52,-3.17,-1.73,0.77,3.07,2.11,4.68,...,4.46,7.30,-1.77,8.00,7.82,3.32,2.63,5.14,-2.41,-1.54
2018-05-01,-3.35,-1.45,1.34,-0.91,-3.54,-2.18,0.28,2.72,1.76,4.35,...,4.18,6.90,-2.13,7.57,7.45,2.93,2.24,4.76,-2.72,-1.88


### Output Files

For Suffolk County, NY, noticed some of the zip codes have exceptional high rent. However, for the home values, as they reflect the middle range of the county, these exceptional high rents distort the returns significantly. Decided to treat the data for Suffolk County, NT, as an outliner, and exclude this from our study.

In [27]:
print(df_returns.shape)
print(df_home_value.shape)
print(df_rental_with_county.shape)

(43, 50)
(44, 50)
(43, 50)


In [28]:
df_returns = df_returns.drop(columns='Suffolk County,NY', axis=1)
df_home_value = df_home_value.drop(columns='Suffolk County,NY', axis=1)
df_rental_with_county = df_rental_with_county.drop(columns='Suffolk County,NY', axis=1)

In [29]:
df_returns.tail()

,"Alameda County,CA","Contra Costa County,CA","Kern County,CA","Los Angeles County,CA","Marin County,CA","Orange County,CA","Placer County,CA","Riverside County,CA","Sacramento County,CA","San Bernardino County,CA",...,"Guadalupe County,TX","Harris County,TX","Hays County,TX","Johnson County,TX","Kaufman County,TX","Montgomery County,TX","Rockwall County,TX","Tarrant County,TX","Travis County,TX","Williamson County,TX"
2021-03-01,-0.68,1.17,4.70,2.00,-0.42,1.19,4.23,6.84,5.02,8.17,...,7.30,8.21,0.14,10.71,11.18,5.33,6.06,7.56,-1.32,0.36
2021-04-01,-1.24,0.84,4.31,1.49,-0.91,0.74,3.78,6.43,4.60,7.46,...,6.79,7.70,-0.52,10.28,10.57,4.89,5.58,7.07,-2.00,-0.37
2021-05-01,-0.93,0.99,4.71,1.95,-0.48,1.19,4.21,6.80,5.05,7.85,...,7.10,8.00,-0.38,10.67,10.75,5.31,5.88,7.31,-1.85,-0.34
2021-06-01,-1.26,0.66,4.52,1.64,-0.75,0.95,3.94,6.54,4.83,7.42,...,6.84,7.70,-0.82,10.48,10.31,5.05,5.55,6.95,-2.36,-0.88
2021-07-01,-1.50,0.64,4.36,1.43,-0.87,0.77,3.72,6.34,4.60,7.25,...,6.58,7.43,-1.25,10.33,9.92,4.87,5.19,6.67,-2.67,-1.33


Make sure the date range ends all together on 202-06-01

In [30]:
df_home_value = df_home_value[df_home_value.index <= '2021-06-01']
df_rental_with_county = df_rental_with_county[df_rental_with_county.index <= '2021-06-01']
df_returns = df_returns[df_returns.index <= '2021-06-01']

In [31]:
print(df_returns.shape)
print(df_home_value.shape)
print(df_rental_with_county.shape)

(42, 49)
(42, 49)
(42, 49)


In [32]:
df_returns.to_csv("./output/returns.csv")
df_home_value.to_csv("./output/home_values.csv")
df_rental_with_county.to_csv("./output/rentals.csv")

In [33]:
get_rate_data().to_csv("./output/mortgage_rate.csv")

### Generate merged data set

In [34]:
df_returns.shape

(42, 49)

In [35]:
df_home_value_m = df_home_value.reset_index().melt(id_vars=['index'], value_vars=df_home_value.columns.values[0:])
df_home_value_m.columns = ['date', 'county', 'selling_price']
df_home_value_m[['county', 'state']] = df_home_value_m['county'].str.split(',', expand=True)

df_rental_with_county_m = df_rental_with_county.reset_index().melt(id_vars = ['index'], value_vars=df_home_value.columns.values[0:])
df_rental_with_county_m.columns = ['date', 'county', 'rent']
df_rental_with_county_m[['county', 'state']] = df_rental_with_county_m['county'].str.split(',', expand=True)

df_returns_m = df_returns.reset_index().melt(id_vars=['index'], value_vars=df_returns.columns.values[0:])
df_returns_m.columns = ['date', 'county', 'roi']
df_returns_m[['county', 'state']] = df_returns_m['county'].str.split(',', expand=True)

In [36]:
print(df_home_value_m.shape)
print(df_rental_with_county_m.shape)
print(df_returns_m.shape)

(2058, 4)
(2058, 4)
(2058, 4)


In [37]:
df_merge = df_home_value_m.merge(df_rental_with_county_m, on=['date', 'county', 'state'])
df_merge = df_merge.merge(df_returns_m, on=['date', 'county', 'state'])
df_merge = df_merge.merge(df_rate, on='date', how='left')

In [38]:
print(df_merge.shape)

(2058, 7)


In [39]:
df_merge.head(3)

,date,county,selling_price,state,rent,roi,rate
0,2018-01-01,Alameda County,840482.0,CA,2541.0,-1.53,0.0395
1,2018-02-01,Alameda County,853351.0,CA,2545.0,-2.32,0.0422
2,2018-03-01,Alameda County,862162.0,CA,2550.0,-2.91,0.0443


In [40]:
df_merge.to_csv('./output/merge.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=718fdf0e-933b-4ec5-90a8-5b2fe887b720' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>